# Simple setup for using gm/id with IHP open PDK

In [4]:
import numpy as np
#for windows
#import sys
#sys.path.append(r'C:\Users\....\gmid') # path to gmid repository
# ------
import matplotlib.pyplot as plt
from mosplot import load_lookup_table, LoadMosfet # make sure that mosplot can be found in the python path
import ipywidgets as widgets
from ipywidgets import interactive
from ipywidgets import interactive_output, HBox, VBox
import matplotlib.ticker as ticker 

In [5]:
pmos_lv_path = '/home/pedersen/projects/IHP-AnalogAcademy/modules/module_0_foundations/gmid_sweeps/pmos_lv_sweep.npy'
nmos_lv_path ='/home/pedersen/projects/IHP-AnalogAcademy/modules/module_0_foundations/gmid_sweeps/nmos_lv_sweep.npy'

lookup_table_pmos = load_lookup_table(pmos_lv_path)
lookup_table_nmos = load_lookup_table(nmos_lv_path)

In [6]:
nmos = LoadMosfet(lookup_table=lookup_table_nmos, mos="nmos", vsb=0.0, vds=0.4)
pmos = LoadMosfet(lookup_table=lookup_table_pmos, mos="pmos", vsb=0, vds=-0.6, vgs=(-1.2, -0.1))

# Function Definitions

In [7]:
def plot_data_vs_data(x_values, y_values, z_values, length, x_axis_name, y_axis_name='y', y_multiplier=1, log=False):
    x_values_flat = np.array(x_values).flatten()
    y_values_flat = np.array(y_values, dtype=np.float64).flatten()
    z_values_flat = np.array(z_values, dtype=np.float64).flatten()
    length_flat = np.array(length).flatten()

    # Ensure all inputs have the same length
    if not (len(x_values_flat) == len(y_values_flat) == len(z_values_flat) == len(length_flat)):
        raise ValueError("All input arrays (x_values, y_values, z_values, length) must have the same number of elements.")

    unique_lengths = np.unique(length_flat)
    unique_lengths_in_micro = unique_lengths * 1e6

    def update_plot(selected_length, x_value=None, y_value=None):
        plt.figure(figsize=(8, 6))  # Ensure the plot is drawn fresh for each update

        if selected_length == "Show All":
            mask = np.ones_like(length_flat, dtype=bool)
        else:
            selected_length_in_micro = float(selected_length.replace(' μm', ''))
            tolerance = 0.1
            # Recalculate the mask with matching shapes
            mask = np.abs(length_flat * 1e6 - selected_length_in_micro) < tolerance

        x_values_for_length = x_values_flat[mask]
        y_values_for_length = y_values_flat[mask] * y_multiplier
        z_values_for_length = z_values_flat[mask]
        length_for_length = length_flat[mask] * 1e6

        if selected_length == "Show All":
            for length_value in np.unique(length_for_length):
                mask_all = (length_for_length == length_value)
                plt.plot(x_values_for_length[mask_all], y_values_for_length[mask_all])

            min_length = np.min(unique_lengths_in_micro)
            max_length = np.max(unique_lengths_in_micro)
            plt.title(f'{y_axis_name} vs {x_axis_name} (Length from {min_length:.2f} μm to {max_length:.2f} μm)')

        else:
            plt.plot(x_values_for_length, y_values_for_length)
            plt.title(f'{y_axis_name} vs {x_axis_name} for {selected_length}')

        plt.xlabel(f'{x_axis_name}')
        plt.ylabel(f'{y_axis_name}')

        if log:
            plt.yscale('log')
            plt.gca().yaxis.set_major_locator(ticker.LogLocator(base=10, subs=[], numticks=10))
            plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f'$10^{int(np.log10(x))}$'))
            plt.ylabel(f'{y_axis_name} (Log Base 10)')

        if y_value is not None and x_value_widget.disabled:
            closest_index = np.abs(y_values_for_length - y_value).argmin()
            closest_x = x_values_for_length[closest_index]
            closest_y = y_values_for_length[closest_index]
            corresponding_z = z_values_for_length[closest_index]

            plt.scatter(closest_x, closest_y, color='blue', label=f'Point ({closest_x:.2f}, {closest_y:.2f})')
            z_value_widget.value = corresponding_z
            print(f"The corresponding {x_axis_name} value for {y_axis_name} = {closest_y:.2f} is: {closest_x:.2f}")
        elif x_value is not None and y_value_widget.disabled:
            closest_index = np.abs(x_values_for_length - x_value).argmin()
            closest_x = x_values_for_length[closest_index]
            closest_y = y_values_for_length[closest_index]
            corresponding_z = z_values_for_length[closest_index]

            plt.scatter(closest_x, closest_y, color='red', label=f'Point ({closest_x:.2f}, {closest_y:.2f})')
            z_value_widget.value = corresponding_z
            print(f"The corresponding {y_axis_name} value for {x_axis_name} = {closest_x:.2f} is: {closest_y:.2f}")

        plt.grid(True)
        plt.legend()
        plt.show()

    dropdown_options = ["Show All"] + [f'{length:.2f} μm' for length in unique_lengths_in_micro]
    length_widget = widgets.Dropdown(
        options=dropdown_options,
        value=dropdown_options[0],
        description='Select Length:',
    )

    x_value_widget = widgets.FloatText(
        value=np.mean(x_values_flat),
        description=f"Select {x_axis_name}:",
        disabled=False
    )

    y_value_widget = widgets.FloatText(
        value=None,
        description=f"Set {y_axis_name}:",
        disabled=True
    )

    z_value_widget = widgets.FloatText(
        value=None,
        description=f"Corresponding z value:",
        disabled=True
    )

    select_x_or_y_widget = widgets.Checkbox(
        value=True,
        description=f"Select {x_axis_name} (uncheck for {y_axis_name})",
    )

    def toggle_x_or_y(change):
        if change['new']:
            x_value_widget.disabled = False
            y_value_widget.disabled = True
        else:
            x_value_widget.disabled = True
            y_value_widget.disabled = False

    select_x_or_y_widget.observe(toggle_x_or_y, names='value')

    output = interactive_output(update_plot, {
        'selected_length': length_widget,
        'x_value': x_value_widget,
        'y_value': y_value_widget
    })

    display(VBox([length_widget, select_x_or_y_widget, HBox([x_value_widget, y_value_widget]), z_value_widget, output]))


def display_resistance(ro_value):
    """Determine the resistance value and its unit."""
    if ro_value < 1e3:
        return ro_value, "Ω"
    elif ro_value < 1e6:
        return ro_value / 1e3, "kΩ"
    elif ro_value < 1e9:
        return ro_value / 1e6, "MΩ"
    else:
        return ro_value / 1e9, "GΩ"

def display_current(Id_value):
    """Determine the current value and its unit."""
    if Id_value < 1e-6:
        return Id_value * 1e9, "nA"  # Convert to nA
    elif Id_value < 1e-3:
        return Id_value * 1e6, "μA"  # Convert to μA
    else:
        return Id_value * 1e3, "mA"   # Convert to mA
    
def dB_to_linear(av_db):
    return 10 ** (av_db / 20)


def determine_inversion_region(gm_id_value, device_type):
    """Determine the inversion region based on gm/id value for NMOS or PMOS."""
    if device_type == 'nmos':
        if gm_id_value > 20:
            return "Weak Inversion"
        elif 10 < gm_id_value <= 20:
            return "Moderate Inversion"
        else:
            return "Strong Inversion"
    elif device_type == 'pmos':
        if gm_id_value > 20:
            return "Weak Inversion"
        elif 10 < gm_id_value <= 20:
            return "Moderate Inversion"
        else:
            return "Strong Inversion"
    else:
        raise ValueError("Invalid device type. Use 'nmos' or 'pmos'.")
    


# Plotting examples

Note: In my cases i will be using my own defined functions to plot more interatively. You can refer to the standard repository to see how to plot using the native functions in the repo...

In [8]:
# Lets start by plotting the intrinsic gain of both pmos and nmos as a function of gm/id. We start by setting the data arrays

id_values_nmos = nmos.extracted_table['id']
gm_values_nmos = nmos.extracted_table['gm']
gds_values_nmos = nmos.extracted_table['gds']
vgs_values_nmos = nmos.extracted_table['vgs']

id_values_pmos = pmos.extracted_table['id']
gm_values_pmos = pmos.extracted_table['gm']
gds_values_pmos = pmos.extracted_table['gds']
vgs_values_pmos = pmos.extracted_table['vgs']


plot_data_vs_data(gm_values_nmos/id_values_nmos, gm_values_nmos/gds_values_nmos, vgs_values_nmos, nmos.extracted_table['lengths'], 'gm/id', 'gds') # plotting nmos data
plot_data_vs_data(gm_values_pmos/id_values_pmos, gm_values_pmos/gds_values_pmos, vgs_values_pmos, pmos.extracted_table['lengths'], 'gm/id', 'gds') # plotting pmos data

# Verifying the models
To ensure that the gmid library is working properly we will simply set the dimensions for a nmos a varify in xschem

In [9]:
gmid = 18 #moderate inversion
id = 1e-6
gm = gmid*id
display_id, unit_id = display_current(id)
print(f'{display_id} {unit_id}')

1.0 μA


# Choosing the channel length by quick overview

Here the focus is on getting a high intrinsic gain

In [10]:
plot_data_vs_data(gm_values_nmos/id_values_nmos, gm_values_nmos/gds_values_nmos, vgs_values_nmos, nmos.extracted_table['lengths'], 'gm/id', 'gds') # plotting nmos data

In [11]:
# By sweeping through the lenghts for a fixed gm/id we see that the lengths is approximatly 3.25e-6
Lnmos = 3.25e-6
gmro = 40.44
ro = gmro/gm
display_ro, unit_ro = display_resistance(ro)
print(f'R_load = {display_ro:.2f} {unit_ro}')

R_load = 2.25 MΩ


# Now we want to find the Corresponding width

In [12]:
width_nmos = nmos.extracted_table['width']
plot_data_vs_data(gm_values_nmos/id_values_nmos, id_values_nmos/width_nmos, vgs_values_nmos, nmos.extracted_table['lengths'], 'gm/id', 'id/W', log=True) # plotting nmos data

In [13]:
# We see that for a gm/id of 18 and length of 3.25 our, id/W is given as 0.3
id_over_width_nmos = 0.3
Wnmos = id/id_over_width_nmos

In [14]:
# Now we can summarize everything

single_transistor_summary = f"""
Width and Length for NMOS
    W = {Wnmos*1e6:.2f} um
    L = {Lnmos*1e6:.2f} um
Inversion Region for NMOS: {determine_inversion_region(gmid, 'nmos')}

Bias Current:
    Id = {display_id:.2f}{unit_id}
Parameters to check
    gm/gds = {gmro:.2f}
    gm = {gm*1e3:.2} mS
    ro = {display_ro:.2f}{unit_ro}

"""
print(single_transistor_summary)


Width and Length for NMOS
    W = 3.33 um
    L = 3.25 um
Inversion Region for NMOS: Moderate Inversion

Bias Current:
    Id = 1.00μA
Parameters to check
    gm/gds = 40.44
    gm = 0.018 mS
    ro = 2.25MΩ


